## Evaluate Trained Reward Function Model and Visualize

In [1]:
import os
import sys

import numpy as np
import pandas as pd

import torch

# get the current script's directory
current_directory = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
# get the parent directory
parent_directory = os.path.dirname(current_directory)
# add the parent directory to the sys.path
sys.path.append(parent_directory)

from utils import constants
from utils.dataset_loader import PolicyDatasetLoader

from optimization.updater import Updater
from optimization.functions import setup_config, get_directories, load_policy_from_path, load_reward_from_path
from optimization.functions import find_indices_of_trajectory_changes, get_estimated_rewards

from models.policy_model import RobotPolicy
from models.reward_model import RewardFunction

In [2]:
pd.set_option("display.max_columns",
              None)

# Initialization

In [3]:
# available evaluating machine
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Evaluating Device: ", device)

# setup hyperparameters
configs = setup_config(device=device)

# create and return preliminary base paths
json_paths, results_path = get_directories(parent_directory=parent_directory,
                                           data_folder_name=constants.TEST_COLLECTION_DATE)

Evaluating Device:  cpu
Current Time:  Feb_06_2024-17_23_03


In [4]:
# load test demonstrations dataset
all_test_data = PolicyDatasetLoader(demo_data_json_paths=json_paths)


================== Policy Dataset Loader ==================

Number of Trajectories:  5
Each Trajectory Length:  30
Full Demo Dataset Size:  163


In [5]:
# get all indice numbers where the new trajectory is initialized in the dataset
trajectory_indices = find_indices_of_trajectory_changes(dataset=all_test_data)

In [6]:
policy_network = RobotPolicy(state_size=configs.state_size,
                             hidden_size=configs.hidden_size,
                             out_size=configs.action_size,
                             log_std_min=configs.policy_log_std_min,
                             log_std_max=configs.policy_log_std_max,
                             log_std_init=configs.policy_log_std_init,
                             device=configs.device)

In [7]:
reward_network = RewardFunction(state_action_size=configs.state_action_size,
                                hidden_size=configs.hidden_size,
                                out_size=configs.reward_size,
                                device=configs.device)

# Functions

# Test

In [8]:
# folder name where policy model parameters are located ("results / policy_network_params / loading_folder_name")
policy_loading_folder_name = constants.POLICY_LOADING_FOLDER
policy_params_name = constants.POLICY_PARAMS_NAME

In [9]:
# folder name where reward model parameters are located ("results / reward_network_params / loading_folder_name")
reward_loading_folder_name = constants.REWARD_LOADING_FOLDER
reward_params_name = constants.REWARD_PARAMS_NAME

In [10]:
# load pretrained policy network parameters
policy_network = load_policy_from_path(policy_network=policy_network,
                                       results_path=results_path,
                                       policy_loading_folder_name=policy_loading_folder_name,
                                       policy_params_name=policy_params_name)

# set policy model to evaluation mode
for param in policy_network.parameters():
    param.requires_grad = False
policy_network = policy_network.eval()

In [11]:
# load pretrained reward network parameters
reward_network = load_reward_from_path(reward_network=reward_network,
                                       results_path=results_path,
                                       reward_loading_folder_name=reward_loading_folder_name,
                                       reward_params_name=reward_params_name)

# set reward model to evaluation mode
for param in reward_network.parameters():
    param.requires_grad = False
reward_network = reward_network.eval()

In [12]:
updater_obj = Updater(configs=configs,
                      policy_network=policy_network,
                      reward_network=reward_network)

In [13]:
nu_factor = torch.tensor(0.0)

## Run Through All Trajectories in a Test Demonstrations

In [14]:
# loop through each separate trajectory inside the testing dataset
for traj_start_index in range(len(trajectory_indices)):
    
    traj_df, reward_values_demo_data, reward_values_estim_data, logprob_action_estim_avg = get_estimated_rewards(configs=configs,
                                                                                                                 updater_obj=updater_obj,
                                                                                                                 data_loader=all_test_data,
                                                                                                                 policy_network=policy_network,
                                                                                                                 reward_network=reward_network,
                                                                                                                 trajectory_indices=trajectory_indices,
                                                                                                                 traj_start_index=traj_start_index,
                                                                                                                 is_inference_reward=True)
    
    irl_train_loss = updater_obj.calculate_irl_loss(demo_traj_reward=reward_values_demo_data,
                                                    robot_traj_reward=reward_values_estim_data,
                                                    log_probability=logprob_action_estim_avg,
                                                    nu_factor=nu_factor)
    
    # store outputs in the dataframe of trajectory information
    traj_df[constants.ACTION_PREDICTION_AVG_LOGPROB_NAME] = logprob_action_estim_avg.numpy().flatten()
    traj_df[constants.REWARD_DEMONSTRATION_TRAJECTORY_NAME] = reward_values_demo_data.numpy().flatten()
    traj_df[constants.REWARD_ROBOT_TRAJECTORY_NAME] = reward_values_estim_data.numpy().flatten()
    

In [15]:
traj_df

,state_label_norm_1,state_label_norm_2,state_label_norm_3,state_label_norm_4,state_label_denorm_1,state_label_denorm_2,state_label_denorm_3,state_label_denorm_4,action_label_norm_1,action_label_norm_2,action_label_norm_3,action_label_denorm_1,action_label_denorm_2,action_label_denorm_3,action_pred_logprob_1,action_pred_logprob_2,action_pred_logprob_3,action_pred_norm_1,action_pred_norm_2,action_pred_norm_3,action_pred_std_1,action_pred_std_2,action_pred_std_3,action_pred_denorm_1,action_pred_denorm_2,action_pred_denorm_3,trajectory_index,state_number,gnll_loss,state_est_norm_1,state_est_norm_2,state_est_norm_3,state_est_norm_4,state_est_denorm_1,state_est_denorm_2,state_est_denorm_3,state_est_denorm_4,next_state_label_norm_1,next_state_label_norm_2,next_state_label_norm_3,next_state_label_norm_4,next_state_label_denorm_1,next_state_label_denorm_2,next_state_label_denorm_3,next_state_label_denorm_4,next_state_est_norm_1,next_state_est_norm_2,next_state_est_norm_3,next_state_est_norm_4,next_state_est_denorm_1,next_state_est_denorm_2,next_state_est_denorm_3,next_state_est_denorm_4,action_pred_avg_logprob,reward_demo_traj,reward_robot_traj
0,0.357050865889,0.838037848473,0.000000000000,0.391351431608,0.714101731777,1.676075696945,0.000000000000,0.782702863216,-0.199934840202,-0.355778723955,0.307092785835,-0.399869680405,-0.711557388306,0.614185571671,1.381720185280,1.382384657860,1.381953120232,-0.254897743464,-0.334094941616,0.310078799725,0.100192822516,0.100126273930,0.100169487298,-0.509795427322,-0.668189883232,0.620157718658,4,0,0.301920861006,0.357050865889,0.838037848473,0.000000000000,0.391351431608,0.714101731777,1.676075696945,0.000000000000,0.782702863216,0.357049099170,0.838033201962,0.000000000000,0.391342785835,0.714098198340,1.676066403923,0.000000000000,0.782685571671,0.340187301223,0.823530480231,0.059160960251,0.394328859329,0.680374602445,1.647060960462,0.118321920502,0.788657718658,1.382019400597,0.998401701450,0.997605681419
1,0.357049137354,0.838033199310,0.000014794530,0.391342788935,0.714098274708,1.676066398621,0.000029589060,0.782685577869,-0.200887247920,-0.356528669596,0.305290669203,-0.401774525642,-0.713057279587,0.610581398010,1.381720185280,1.382384657860,1.381953120232,-0.298301160336,-0.302041709423,0.310738950968,0.100192822516,0.100126273930,0.100169487298,-0.596602320671,-0.604083418846,0.621477842331,4,1,0.305598944426,0.340187311172,0.823530495167,0.059160958976,0.394328862429,0.680374622345,1.647060990334,0.118321917951,0.788657724857,0.357650418535,0.837880037692,0.002171875745,0.389540699005,0.715300837071,1.675760075385,0.004343751490,0.779081398010,0.319422333527,0.801272564654,0.112146737658,0.394988921165,0.638844667054,1.602545129308,0.224293475315,0.789977842331,1.382019400597,0.998550713062,0.988874316216
2,0.357650429010,0.837880015373,0.002183987992,0.389540672302,0.715300858021,1.675760030746,0.004367975984,0.779081344604,-0.213343515992,-0.352634400129,0.306205183268,-0.426687002182,-0.705268859863,0.612410306931,1.381720185280,1.382384657860,1.381953120232,-0.334620386362,-0.264129817486,0.309306263924,0.100192822516,0.100126273930,0.100169487298,-0.669240713120,-0.528259634972,0.618612527847,4,2,0.309716969728,0.319422334433,0.801272571087,0.112146735191,0.394988924265,0.638844668865,1.602545142174,0.224293470383,0.789977848530,0.354093252101,0.835267363275,0.013800959226,0.390455153465,0.708186504202,1.670534726551,0.027601918451,0.780910306931,0.298212836032,0.774702205680,0.162925149673,0.393556263924,0.596425672063,1.549404411360,0.325850299346,0.787112527847,1.382019400597,0.998427450657,0.949690937996
3,0.354093223810,0.835267364979,0.013813394122,0.390455186367,0.708186447620,1.670534729958,0.027626788244,0.780910372734,-0.230947449803,-0.341997474432,0.305424094200,-0.461894869804,-0.683995008469,0.610848188400,1.381720185280,1.382384657860,1.381953120232,-0.368749916553,-0.222677960992,0.305788040161,0.100192822516,0.100126273930,0.100169487298,-0.737499833107,-0.44